<a href="https://colab.research.google.com/github/Karthik-Kundurthy/pws-soundprofiler-cv/blob/main/pws_soundprofiler_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git config --global user.email ""

In [ ]:
!git config --global user.name ""

In [ ]:
! git clone https://[token]@github.gatech.edu/Aquabots-VIP/pws-soundprofiler-ai

Cloning into 'pws-soundprofiler-ai'...
remote: Enumerating objects: 39447, done.
remote: Counting objects: 100% (1618/1618), done.
remote: Compressing objects: 100% (1580/1580), done.
remote: Total 39447 (delta 44), reused 1549 (delta 32), pack-reused 37829
Receiving objects: 100% (39447/39447), 867.27 MiB | 34.69 MiB/s, done.
Resolving deltas: 100% (170/170), done.
Checking out files: 100% (37782/37782), done.


In [ ]:
%cd pws-soundprofiler-ai

/content/pws-soundprofiler-ai


In [ ]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from data_generator import DataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dropout, Dense, BatchNormalization
from tensorflow.keras.layers import GlobalAveragePooling2D, Concatenate
from tensorflow.keras.utils import to_categorical
import pickle


In [ ]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
#Single CLS
n_classes = 43
input_shape = (299, 299, 3)
feat_shape = (16,)

class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs', **kwargs):
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'inception_v3_training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'inception_v3_validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.create_file_writer(self.val_log_dir)       
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        # for name, value in val_logs.items():
        #     summary = tf.Summary()
        #     summary_value = summary.value.add()
        #     summary_value.simple_value = value.item()
        #     summary_value.tag = name
        #     self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

def get_single_cls_model():

    pretrain_model = tf.keras.applications.InceptionV3(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape)

    input_image = Input(shape=input_shape)
    x = pretrain_model(input_image)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = BatchNormalization()(x)
    c1 = Dense(256-feat_shape[0], activation='relu')(x)
    c2 = Input(shape=feat_shape)
    c = Concatenate(axis=-1,)([c1, c2])
    x = BatchNormalization()(c)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    output = Dense(n_classes, activation='softmax')(x)

    model = Model([input_image, c2], output)
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(1e-3),
                  metrics=['acc'])
    model.summary()
    return model

def get_single_cls_model_custom(pretrain_model, dense_dimension=512, lr=1e-3):
    for layer in pretrain_model.layers:
      layer.trainable = False

    input_image = Input(shape=input_shape)
    x = pretrain_model(input_image)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)

    x = Dense(dense_dimension, activation='relu')(x)

    x = BatchNormalization()(x)
    c1 = Dense(256-feat_shape[0], activation='relu')(x)
    c2 = Input(shape=feat_shape)
    c = Concatenate(axis=-1,)([c1, c2])
    x = BatchNormalization()(c)
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    output = Dense(n_classes, activation='softmax')(x)

    model = Model([input_image, c2], output)
    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr),
                  metrics=['acc'])
    model.summary()
    return model


def check_dirs(dirs):
    for d in dirs:
        exists = os.path.join(os.getcwd(), d)
        if os.path.isdir(exists) is False:
            os.mkdir(exists)

In [ ]:
print('Checking dirs')
check_dirs(['logs', 'models'])
print('Loading class map')
with open('class_map.p', 'rb') as handle:
    class_map = pickle.load(handle)
print('Loading plankton.csv')
df = pd.read_csv('plankton.csv')
df.drop_duplicates(subset='im_name', inplace=True, keep=False)

params = {'n_classes': n_classes,
          'shape': input_shape,
          'feat_shape': feat_shape,
          'batch_size': 64,
          'shuffle': True}

frames = []
for c in np.unique(df.label):
    frames.append(df[df.label==c].sample(n=5000, replace=True, random_state=0))
df_sample = pd.concat(frames)

paths = []
labels = []
data_path = os.path.join(os.getcwd(), 'pad')
print('Creating labelled images')
for im_name, label in zip(df_sample.im_name, df_sample.label):
    im_dir = os.path.join(data_path, class_map[label])
    im_path = os.path.join(im_dir, im_name)
    paths.append(im_path)
    labels.append(to_categorical(y=label, num_classes=n_classes))

paths = np.array(paths)
labels = np.array(labels)

X_train, X_val, y_train, y_val = train_test_split(paths, labels, test_size=0.1, random_state=0)

checkpoint = ModelCheckpoint('./models/inception_v3.model', monitor='val_acc', verbose=1, mode='max',
                             save_best_only=True, save_weights_only=False, period=1)

reduceLROnPlato = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                    patience=3, verbose=1, mode='min')

tensorboard = TrainValTensorBoard(write_graph=False)

Checking dirs
Loading class map
Loading plankton.csv
Creating labelled images


## Insert Model Template

In [ ]:
#Insert your own model here. For a list, visit https://keras.io/api/applications/ 
pretrain_model = tf.keras.applications.'''Put your own net here'''(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape)

#Change the below line
model = get_single_cls_model_custom(pretrain_model, 1e-3)

tg = DataGenerator(paths=X_train, labels=y_train, augment=True, **params)
vg = DataGenerator(paths=X_val, labels=y_val, **params)
#model = load_model('./models/inception_v3.model')
print('Training model:')
hist=model.fit_generator(generator=tg, validation_data=vg,
                    steps_per_epoch=len(tg)/10, validation_steps=len(vg),
                    epochs=10, verbose=1,
                    callbacks=[tensorboard, checkpoint])

SyntaxError: ignored

## EfficientNetB3

In [ ]:
#Insert your own model here. For a list, visit https://keras.io/api/applications/ 
pretrain_model = tf.keras.applications.EfficientNetB3(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape)

#Change the below line
model = get_single_cls_model_custom(pretrain_model, 1e-3)

tg = DataGenerator(paths=X_train, labels=y_train, augment=True, **params)
vg = DataGenerator(paths=X_val, labels=y_val, **params)
#model = load_model('./models/inception_v3.model')
print('Training model:')
hist=model.fit_generator(generator=tg, validation_data=vg,
                    steps_per_epoch=len(tg)/10, validation_steps=len(vg),
                    epochs=10, verbose=1,
                    callbacks=[tensorboard, checkpoint])

## Xception

In [ ]:
#Insert your own model here. For a list, visit https://keras.io/api/applications/ 
pretrain_model = tf.keras.applications.Xception(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape)

#Change the below line
model = get_single_cls_model_custom(pretrain_model, 1e-3)

tg = DataGenerator(paths=X_train, labels=y_train, augment=True, **params)
vg = DataGenerator(paths=X_val, labels=y_val, **params)
#model = load_model('./models/inception_v3.model')
print('Training model:')
hist=model.fit_generator(generator=tg, validation_data=vg,
                    steps_per_epoch=len(tg)/10, validation_steps=len(vg),
                    epochs=10, verbose=1,
                    callbacks=[tensorboard, checkpoint])

 4202496/83683744 [>.............................] - ETA: 1s

KeyboardInterrupt: ignored

## ResNet50

In [ ]:
#Insert your own model here. For a list, visit https://keras.io/api/applications/ 
pretrain_model = tf.keras.applications.ResNet50(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape)

#Change the below line
model = get_single_cls_model_custom(pretrain_model, 1e-3)

tg = DataGenerator(paths=X_train, labels=y_train, augment=True, **params)
vg = DataGenerator(paths=X_val, labels=y_val, **params)
#model = load_model('./models/inception_v3.model')
print('Training model:')
hist=model.fit_generator(generator=tg, validation_data=vg,
                    steps_per_epoch=len(tg)/10, validation_steps=len(vg),
                    epochs=10, verbose=1,
                    callbacks=[tensorboard, checkpoint])




## DenseNet121

In [ ]:
#Insert your own model here. For a list, visit https://keras.io/api/applications/ 
pretrain_model = tf.keras.applications.DenseNet121(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape)

#Change the below line
model = get_single_cls_model_custom(pretrain_model, 1e-3)

tg = DataGenerator(paths=X_train, labels=y_train, augment=True, **params)
vg = DataGenerator(paths=X_val, labels=y_val, **params)
#model = load_model('./models/inception_v3.model')
print('Training model:')
hist=model.fit_generator(generator=tg, validation_data=vg,
                    steps_per_epoch=len(tg)/10, validation_steps=len(vg),
                    epochs=10, verbose=1,
                    callbacks=[tensorboard, checkpoint])

29084464/29084464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 densenet121 (Functional)       (None, 9, 9, 1024)   7037504     ['input_2[0][0]']                
                                                                                                  
 global_average_pooling2d (Glob  (None, 1024)        0           ['densenet121[0][0]']            
 alAveragePooling2D)                                                                              
                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
303/302 [==============================] - ETA: 0s - loss: 2.1291 - acc: 0.4216
Epoch 1: val_acc improved from -inf to 0.61744, saving model to ./models/inception_v3.model


302/302 [==============================] - 283s 875ms/step - loss: 2.1291 - acc: 0.4216 - val_loss: 1.3015 - val_acc: 0.6174
Epoch 2/10
303/302 [==============================] - ETA: 0s - loss: 1.4513 - acc: 0.5778
Epoch 2: val_acc improved from 0.61744 to 0.66670, saving model to ./models/inception_v3.model


302/302 [==============================] - 299s 991ms/step - loss: 1.4513 - acc: 0.5778 - val_loss: 1.0951 - val_acc: 0.6667


KeyboardInterrupt: ignored




## Efficient Net V2L


In [ ]:
#Insert your own model here. For a list, visit https://keras.io/api/applications/ 
pretrain_model = tf.keras.applications.EfficientNetV2L(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape)


#Change the below line
model = get_single_cls_model_custom(pretrain_model, 1e-3)

tg = DataGenerator(paths=X_train, labels=y_train, augment=True, **params)
vg = DataGenerator(paths=X_val, labels=y_val, **params)
#model = load_model('./models/inception_v3.model')
print('Training model:')
hist=model.fit_generator(generator=tg, validation_data=vg,
                    steps_per_epoch=len(tg)/10, validation_steps=len(vg),
                    epochs=10, verbose=1,
                    callbacks=[tensorboard, checkpoint])

473176280/473176280 [==============================] - 15s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 efficientnetv2-l (Functional)  (None, 10, 10, 1280  117746848   ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 global_average_pooling2d (Glob  (None, 1280)        0           ['efficientnetv2-l[0][0]']       
 alAveragePooling2D)      

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
303/302 [==============================] - ETA: 0s - loss: 3.2099 - acc: 0.1514
Epoch 1: val_acc improved from -inf to 0.25634, saving model to ./models/inception_v3.model


302/302 [==============================] - 1158s 4s/step - loss: 3.2099 - acc: 0.1514 - val_loss: 2.8760 - val_acc: 0.2563
Epoch 2/10
303/302 [==============================] - ETA: 0s - loss: 2.5345 - acc: 0.2905
Epoch 2: val_acc improved from 0.25634 to 0.37332, saving model to ./models/inception_v3.model


302/302 [==============================] - 1116s 4s/step - loss: 2.5345 - acc: 0.2905 - val_loss: 2.2231 - val_acc: 0.3733
Epoch 3/10
183/302 [=================>............] - ETA: 2:46 - loss: 2.2970 - acc: 0.3408

In [ ]:
#Insert your own model here. For a list, visit https://keras.io/api/applications/ 
pretrain_model = tf.keras.applications.InceptionV3(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape)

#Change the below line
model = get_single_cls_model_custom(pretrain_model, 1, 1e-3)

tg = DataGenerator(paths=X_train, labels=y_train, augment=True, **params)
vg = DataGenerator(paths=X_val, labels=y_val, **params)
#model = load_model('./models/inception_v3.model')
print('Training model:')
hist=model.fit_generator(generator=tg, validation_data=vg,
                    steps_per_epoch=len(tg)/10, validation_steps=len(vg),
                    epochs=10, verbose=1,
                    callbacks=[tensorboard, checkpoint])

87910968/87910968 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 inception_v3 (Functional)      (None, 8, 8, 2048)   21802784    ['input_2[0][0]']                
                                                                                                  
 global_average_pooling2d (Glob  (None, 2048)        0           ['inception_v3[0][0]']           
 alAveragePooling2D)                                                                              
                             

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/10
303/302 [==============================] - ETA: 0s - loss: 2.8655 - acc: 0.2251
Epoch 1: val_acc improved from -inf to 0.16875, saving model to ./models/inception_v3.model


302/302 [==============================] - 273s 854ms/step - loss: 2.8655 - acc: 0.2251 - val_loss: 3.1024 - val_acc: 0.1688
Epoch 2/10
303/302 [==============================] - ETA: 0s - loss: 2.2897 - acc: 0.3514
Epoch 2: val_acc improved from 0.16875 to 0.34888, saving model to ./models/inception_v3.model


302/302 [==============================] - 248s 822ms/step - loss: 2.2897 - acc: 0.3514 - val_loss: 2.2578 - val_acc: 0.3489
Epoch 3/10
303/302 [==============================] - ETA: 0s - loss: 2.1167 - acc: 0.3903

## Mobile Net V2


In [ ]:
#Insert your own model here. For a list, visit https://keras.io/api/applications/ 
pretrain_model = tf.keras.applications.MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape)

#Change the below line
model = get_single_cls_model_custom(pretrain_model, 1e-3)

tg = DataGenerator(paths=X_train, labels=y_train, augment=True, **params)
vg = DataGenerator(paths=X_val, labels=y_val, **params)
#model = load_model('./models/inception_v3.model')
print('Training model:')
hist=model.fit_generator(generator=tg, validation_data=vg,
                    steps_per_epoch=len(tg)/10, validation_steps=len(vg),
                    epochs=10, verbose=1,
                    callbacks=[tensorboard, checkpoint])

9406464/9406464 [==============================] - 0s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 mobilenetv2_1.00_224 (Function  (None, 10, 10, 1280  2257984    ['input_3[0][0]']                
 al)                            )                                                                 
                                                                                                  
 global_average_pooling2d (Glob  (None, 1280)        0           ['mobilenetv2_1.00_224[0][0]']   
 alAveragePooling2D)           

<ipython-input-12-b8f2ba520383>:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist=model.fit_generator(generator=tg, validation_data=vg,


Epoch 1/10
303/302 [==============================] - ETA: 0s - loss: 2.7445 - acc: 0.2623
Epoch 1: val_acc improved from -inf to 0.14953, saving model to ./models/inception_v3.model


302/302 [==============================] - 214s 673ms/step - loss: 2.7445 - acc: 0.2623 - val_loss: 3.0791 - val_acc: 0.1495
Epoch 2/10
303/302 [==============================] - ETA: 0s - loss: 2.2356 - acc: 0.3671
Epoch 2: val_acc improved from 0.14953 to 0.39520, saving model to ./models/inception_v3.model


302/302 [==============================] - 200s 661ms/step - loss: 2.2356 - acc: 0.3671 - val_loss: 2.1836 - val_acc: 0.3952
Epoch 3/10
303/302 [==============================] - ETA: 0s - loss: 2.1099 - acc: 0.3944
Epoch 3: val_acc improved from 0.39520 to 0.41063, saving model to ./models/inception_v3.model


...

302/302 [==============================] - 204s 674ms/step - loss: 1.9629 - acc: 0.4268 - val_loss: 1.9063 - val_acc: 0.4411
Epoch 6/10
303/302 [==============================] - ETA: 0s - loss: 1.9334 - acc: 0.4317
Epoch 6: val_acc improved from 0.44114 to 0.45686, saving model to ./models/inception_v3.model


302/302 [==============================] - 202s 669ms/step - loss: 1.9334 - acc: 0.4317 - val_loss: 1.8406 - val_acc: 0.4569
Epoch 7/10
303/302 [==============================] - ETA: 0s - loss: 1.8969 - acc: 0.4361
Epoch 7: val_acc did not improve from 0.45686
302/302 [==============================] - 184s 609ms/step - loss: 1.8969 - acc: 0.4361 - val_loss: 1.8519 - val_acc: 0.4538
Epoch 8/10
303/302 [==============================] - ETA: 0s - loss: 1.8653 - acc: 0.4454
Epoch 8: val_acc improved from 0.45686 to 0.46315, saving model to ./models/inception_v3.model


302/302 [==============================] - 200s 663ms/step - loss: 1.8653 - acc: 0.4454 - val_loss: 1.8159 - val_acc: 0.4632
Epoch 9/10
303/302 [==============================] - ETA: 0s - loss: 1.8343 - acc: 0.4528
Epoch 9: val_acc improved from 0.46315 to 0.48372, saving model to ./models/inception_v3.model


302/302 [==============================] - 199s 660ms/step - loss: 1.8343 - acc: 0.4528 - val_loss: 1.7362 - val_acc: 0.4837
Epoch 10/10
303/302 [==============================] - ETA: 0s - loss: 1.8101 - acc: 0.4538
Epoch 10: val_acc improved from 0.48372 to 0.49109, saving model to ./models/inception_v3.model


302/302 [==============================] - 200s 663ms/step - loss: 1.8101 - acc: 0.4538 - val_loss: 1.7038 - val_acc: 0.4911
